In [122]:
import keras
from keras import layers

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt


In [124]:
raw_data = pd.read_csv("btc_minute_data_2025", index_col="Unnamed: 0")

In [126]:
TIMESTEP = 60

def make_convolutions(r_data, timestep=TIMESTEP, target_shift = 1, mode = "linear"):
    output = []
    target = []
    if mode == "linear":
        data = (r_data - np.min(r_data))/(np.max(r_data)-np.min(r_data))
    else:
        data = (r_data - np.mean(r_data))/np.sd(r_data)
    for i in range(timestep, len(data)-target_shift):
        output.append(data[(i-timestep):(i)])
        target.append(data[i + target_shift])
    return np.stack(output), np.stack(target)
                      

def train_test_split(X, y, split_ratio = 0.8):
    if len(X) != len(y):
        return print("Error, invalid lengths: ", len(X), " len X does not match len y ", len(y))
    else:
        split_index = round(split_ratio * len(X))
        return X[:split_index], y[:split_index], X[split_index:], y[split_index:]

np_raw = raw_data.values.reshape(-1, 1)
windows_raw, target_raw = make_convolutions(np_raw)

X_train, y_train, X_test, y_test = train_test_split(windows_raw, target_raw)

In [127]:
X_train.shape

(839697, 60, 1)

In [130]:
model = keras.Sequential(
    [
        layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
        layers.Conv1D(
            filters = 32, 
            kernel_size=60,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Conv1DTranspose(
            filters = 32, 
            kernel_size=60,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Dropout(rate = 0.2),
        
        layers.Conv1D(
            filters = 16,
            kernel_size=60,
            padding="same",
            strides=2,
            activation="relu",
        ),
        layers.Conv1DTranspose(
            filters = 16,    #beudeutet pro layer sliden 32 1D Filter über den Datensatz
                             #gleichzusetzen mit Dimension Erweiterung in normalen NN, pro 1 Input, werden 32 outputs erzeugt        
            kernel_size=60,  #ein conv1D besteht aus 7 weights, added toegther
            padding="same",  #Padding um Output Length = Input legth, kein loss am Rand
            strides=2,       #macht immer 2 Schritte, halbiert tatsächlichen Output
            activation="relu",
            ),
        layers.Dropout(rate = 0.2),
            #layers.Conv1DTranspose(filters = 1, kernel_size=7, padding="same"),
        layers.Flatten(),
        layers.Dense(32, activation="relu"),
        layers.Dense(1)
            ]
)

model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.001), loss = "mse")
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_8 (Conv1D)               │ (None, 30, 32)         │         1,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_5              │ (None, 60, 32)         │        61,472 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 60, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (None, 30, 16)         │        30,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_transpose_6              │ (None, 60, 16)         │        15,376 │
│ (Conv1DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 60, 16)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 960)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │        30,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 140,321 (548.13 KB)

 Trainable params: 140,321 (548.13 KB)

 Non-trainable params: 0 (0.00 B)

In [132]:
model.fit(
    X_train, 
    y_train,
    epochs = 12,
    batch_size=32,
    validation_split=0.15,
    callbacks = [
        keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10, mode="min")
    ]
)

Epoch 1/12
22305/22305 ━━━━━━━━━━━━━━━━━━━━ 141s 6ms/step - loss: 2.4137e-04 - val_loss: 4.2770e-05
Epoch 2/12
22305/22305 ━━━━━━━━━━━━━━━━━━━━ 139s 6ms/step - loss: 2.0172e-05 - val_loss: 2.6102e-05
Epoch 3/12
22305/22305 ━━━━━━━━━━━━━━━━━━━━ 141s 6ms/step - loss: 1.4713e-05 - val_loss: 2.4716e-05
Epoch 4/12
22305/22305 ━━━━━━━━━━━━━━━━━━━━ 133s 6ms/step - loss: 1.2678e-05 - val_loss: 1.4676e-05
Epoch 5/12
22305/22305 ━━━━━━━━━━━━━━━━━━━━ 146s 7ms/step - loss: 1.1826e-05 - val_loss: 6.7956e-05
Epoch 6/12
22305/22305 ━━━━━━━━━━━━━━━━━━━━ 138s 6ms/step - loss: 1.0804e-05 - val_loss: 2.2885e-05
Epoch 7/12
22305/22305 ━━━━━━━━━━━━━━━━━━━━ 142s 6ms/step - loss: 1.0029e-05 - val_loss: 1.0282e-05
Epoch 8/12
22305/22305 ━━━━━━━━━━━━━━━━━━━━ 146s 7ms/step - loss: 9.1677e-06 - val_loss: 1.9706e-05
Epoch 9/12
22305/22305 ━━━━━━━━━━━━━━━━━━━━ 143s 6ms/step - loss: 9.6342e-06 - val_loss: 8.3915e-06
Epoch 10/12
22305/22305 ━━━━━━━━━━━━━━━━━━━━ 143s 6ms/step - loss: 8.7507e-06 - val_loss: 2.9920e-05

In [56]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.show()

[[1, 2], [3, 4], [5, 6]]
[]
